In [2]:
from astropy import units as u
from astropy import time

import numpy as np
import math as Math
from poliastro import iod
from poliastro.bodies import Mars, Earth, Sun
from poliastro.ephem import Ephem
from poliastro.maneuver import Maneuver
from poliastro.twobody import Orbit
from poliastro.util import time_range
from poliastro.plotting import OrbitPlotter3D, StaticOrbitPlotter
import math

# More info: https://plotly.com/python/renderers/
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

from astropy.coordinates import solar_system_ephemeris


solar_system_ephemeris.set("jpl")

c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\poliastro\core\_jit.py:29: UserWarning: Could not import numba package. All poliastro functions will work properly but the CPU intensive algorithms will be slow. Consider installing numba to boost performance.
  warnings.warn(


<ScienceState solar_system_ephemeris: 'jpl'>

In [3]:
#launch from body_1 to body_2
body_1 = Earth
body_2 = Mars


#solve for a minimum ejection DV given a time period
transitTime = 90*u.day
resolution = 10*u.day
start_date = time.Time("2035-07-15 00:01", scale="utc").tdb
end_date = time.Time("2035-10-13 00:01", scale="utc").tdb

ephem_1 = Ephem.from_body(body_1, time_range(start_date-transitTime, end=end_date+transitTime))
ephem_2 = Ephem.from_body(body_2, time_range(start_date-transitTime, end=end_date+transitTime))


ss_planet_1 = Orbit.from_ephem(Sun, ephem_1, start_date)
ss_planet_2 = Orbit.from_ephem(Sun, ephem_2, end_date)


#interpolate a trajectory that takes transitTime
#t in days value
def sun_D(t):
    return (np.linalg.norm(ss_planet_2.r)-np.linalg.norm(ss_planet_1.r))*Math.sin(Math.pi*t/(180))+1*np.linalg.norm(ss_planet_1.r)

print(sun_D(90))


#assume 0.6 albedo
albedo = 0.6
starship_area = 9**2*Math.pi #starship nosecone facing sun only
base_flux = 1.381*(sun_D(0)**2)#KWm^2, base_flux/sun_D(0)^2 = 1.381

def flux_calc(t):
    return base_flux/(sun_D(t)**2)

#integrate over the 90 day transit in 0.1 day steps
power_ab = 0
for i in range(900):
    power_ab += (1-albedo)*flux_calc(i/10)*8640*starship_area
print(f"power absorbed is {power_ab} kJ")
#lox latent heat is 214kJ
print(f"Thus can boil {power_ab/(1000*214)} tonnes of LoX")


##332+-23 uGy/day
#1.84+-0.33mSv/day
rad_day = 1.84
error = 0.33
print(f"Total dosage for one transit is {rad_day*transitTime.value}mSv")
print(f"+-{int(error*transitTime.value)}mSv")



208941892.47326568 km
power absorbed is 737473788.1539918 kJ
Thus can boil 3446.1391969812703 tonnes of LoX
Total dosage for one transit is 165.6mSv
+-29mSv


c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\erfa\core.py:133: ErfaWarning:

ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"

c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\erfa\core.py:133: ErfaWarning:

ERFA function "utctai" yielded 1 of "dubious year (Note 3)"

c:\Users\jacki\miniforge-pypy3\envs\python_3.9\lib\site-packages\erfa\core.py:133: ErfaWarning:

ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"



In [7]:
#block 1 specs

#6 rings lox tank
#3 rings ch4 tank
#https://www.transmetra.ch/images/transmetra_pdf/publikationen_literatur/pyrometrie-thermografie/emissivity_table.pdf steel
#https://ntrs.nasa.gov/api/citations/19810003644/downloads/19810003644.pdf tiles

#thus
ring_area = 1.83*2*Math.pi*9
steel_emissivity = 0.44#depends on heating
tile_emissivity = 0.8

bolz = 5.67*(10**-8)

rad_ch4 = 3*ring_area*5.67*(10**-8)*(0.44+0.8)*0.5*(140**4)
rad_lox = 6*ring_area*5.67*(10**-8)*(0.44+0.8)*0.5*(113**4)
print(f"Ring area is {ring_area} m^2")
print(f"Radiated heat frm tanks is {int(rad_ch4+rad_lox)} W")
#consider polished aluminium plate at bottom of crew cabin
cabin_power = Math.pi*(4.5**2)*0.06*bolz*(290**4)
print(f"Radiated cabin heat is {int(cabin_power)} W")


therm_grad = 1.8/200
power_conduct = 0.004*2*Math.pi*9*15/therm_grad
print(f"Conducted power is {power_conduct} W")

#https://ntrs.nasa.gov/api/citations/19650024830/downloads/19650024830.pdf steel conducitivty

##hopefully not needed
#Extra sun 
#sun disc is https://web.archive.org/web/20100527142627/http://education.gsfc.nasa.gov/eclipse/pages/faq.html
#0.53 deg






Ring area is 103.4840620092478 m^2
Radiated heat frm tanks is 7751 W
Radiated cabin heat is 1530 W
Conducted power is 339.29200658769764 W
